<a href="https://colab.research.google.com/github/EfeTascioglu/hackTheNorth2019/blob/master/QuakeTester.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
import math
import requests
import datetime as dt
from datetime import datetime
import sklearn as sk
from sklearn.model_selection import train_test_split
from datetime import datetime

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import preprocessing
import matplotlib.pyplot as plt
%matplotlib inline

In [0]:
!wget https://github.com/efetascioglu/hackTheNorth2019/raw/master/query1.csv

In [0]:
df = pd.read_csv('query1.csv') # (Dataframe) 

In [141]:
df.head()

,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,timeYear,timeMonth,timeWeek,timeDay,timeDayofweek,timeDayofyear,timeIs_month_end,timeIs_month_start,timeIs_quarter_end,timeIs_quarter_start,timeIs_year_end,timeIs_year_start,timeElapsed
0,54.5263,-35.1239,10.00,5.0,mb,NaN,70.0,9.637,1.20,us,us70005gh4,2019-09-14T21:29:19.040Z,Reykjanes Ridge,earthquake,9.4,1.9,0.025,493.0,reviewed,us,us,2019,9,37,14,5,257,False,False,False,False,False,False,1568495452
1,-30.2566,-177.8124,10.00,5.1,mb,NaN,161.0,1.013,1.25,us,us70005ge6,2019-09-14T17:01:57.040Z,"110km S of Raoul Island, New Zealand",earthquake,13.4,1.9,0.106,29.0,reviewed,us,us,2019,9,37,14,5,257,False,False,False,False,False,False,1568479327
2,-0.9714,128.5726,10.00,5.9,mww,NaN,42.0,2.110,0.78,us,us70005ge3,2019-09-14T18:23:50.032Z,"108km ENE of Laiwui, Indonesia",earthquake,6.6,1.8,0.061,26.0,reviewed,us,us,2019,9,37,14,5,257,False,False,False,False,False,False,1568478088
3,-33.9419,-73.2487,10.00,4.7,mb,NaN,152.0,1.629,0.87,us,us70005gde,2019-09-14T15:42:21.040Z,"155km WSW of San Antonio, Chile",earthquake,6.3,1.9,0.080,47.0,reviewed,us,us,2019,9,37,14,5,257,False,False,False,False,False,False,1568472052
4,53.9819,-165.9177,73.06,5.4,mww,NaN,49.0,0.176,1.11,us,us70005gcr,2019-09-15T01:36:03.274Z,"19km SSW of Akutan, Alaska",earthquake,6.5,3.5,0.048,41.0,reviewed,us,us,2019,9,37,14,5,257,False,False,False,False,False,False,1568467555


In [0]:
df = df.drop('id', axis=1)
df = df.drop('net', axis=1)
df = df.drop('magType', axis=1)
df = df.drop('nst', axis=1)
df = df.drop('type', axis=1)
df = df.drop('status', axis=1)
df = df.drop('locationSource', axis=1)
df = df.drop('magSource', axis=1)
df = df.drop('updated', axis=1)
df = df.drop('depthError', axis=1)
df = df.drop('magError', axis=1)


#df = df.drop('time', axis=1)

In [0]:
df.dropna()
df.head()

In [0]:
from fastai.tabular.transform import add_datepart
add_datepart(df, 'time') # inplace
df.head()

In [0]:
df = df.drop('time', axis=1) 	 	 	 	 	 	 	 	 	 	
df = df.drop('timeYear', axis=1)
df = df.drop('timeMonth', axis=1)
df = df.drop('timeWeek', axis=1)
df = df.drop('timeDay', axis=1)
df = df.drop('timeDayofweek', axis=1)
df = df.drop('timeDayofyear', axis=1)
df = df.drop('timeIs_month_end', axis=1)
df = df.drop('timeIs_month_start', axis=1)
df = df.drop('timeIs_quarter_end', axis=1)
df = df.drop('timeIs_quarter_start', axis=1)
df = df.drop('timeIs_year_end', axis=1)
df = df.drop('timeIs_year_start', axis=1)


In [0]:
#y_train, y_test = pd.DataFrame(train.iloc[:, 8]), pd.DataFrame(test.iloc[:, 8])
#x_train = pd.DataFrame(train.iloc[:, :8] + train.iloc[:, :8])
#x_test = pd.DataFrame(test[['Latitude', 'Longitude']])

In [0]:
df.info()

In [0]:
#y = pd.DataFrame(df.iloc[:, 4])
#x = pd.DataFrame((df.iloc[:, :4]) + (df.iloc[:, 5:8]) + (df.iloc[:, 9:]))
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


In [0]:
#df['time'] = pd.to_datetime(df['time'])

In [0]:
y = pd.DataFrame(df[["mag"]])
#x = pd.DataFrame(df[["latitude", "longitude", "depth", "gap", "dmin", "rms", "horizontalError", "magNst",  "timeElapsed"]])
x = pd.DataFrame(df[["latitude", "longitude", "depth", "gap", "timeElapsed"]])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [0]:
#y = pd.DataFrame(df.iloc[:, 4])
#x = pd.DataFrame((df.iloc[:, :4]) + (df.iloc[:, 5:]))
#x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)


In [0]:
#x_train['time'] = pd.to_datetime(x_train['time'])#.dt.tz_convert('US/Eastern')
#x_test['time']  = pd.to_datetime(x_test['time'])#.dt.tz_convert('US/Eastern')

In [0]:
x_train.head()

In [0]:
x_train['timeElapsed'].unique()

In [0]:
df = pd.Series([x_train])
df.map({np.nan: 0})
df = x_train.DataFrame()

In [0]:
#train_x, test, test = train_test_split(df, test_size = 0.2)

In [0]:
x_train.info()

In [0]:
d_train = xgb.DMatrix(data=x_train, label=y_train) # Everything (Train)
d_test = xgb.DMatrix(data = x_test, label=y_test)

Scikit Learn
XGBoost
MatPlotLib

In [0]:
params = {'booster': 'gbtree', 
          'objective': 'single:squarrederror',
          'colsample_bytree': 1,
          'learning_rate': 0.2,
          'max_depth': 4,
          'lambda': 3,
          'num_class': 7}


xgb_model = xgb.train(params=params,
                      dtrain=d_train,
                      evals=[(d_train, 'train'), (d_test, 'eval')],
                      num_boost_round=100)


In [0]:
model = XGBClassifier()
model.fit(x_train, y_train)
print(model)

In [0]:
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [0]:
reg =  linear_model.LinearRegression()
reg.fit(x_train, y_train)
y_predict = regr.predict(x_test)

In [0]:
#for i in range (5):
#  x_train[i].fillna(0, inplace=True)

In [0]:
from sklearn import preprocessing

x = x_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_train = pd.DataFrame(x_scaled)

In [0]:

x = y_train.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
y_train = pd.DataFrame(x_scaled)

In [0]:
from sklearn import preprocessing

x = x_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
x_test = pd.DataFrame(x_scaled)

In [0]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [0]:
from sklearn import preprocessing

x = y_test.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
y_test = pd.DataFrame(x_scaled)

In [0]:
x_train.head

In [0]:
#X, y = make_classification(n_samples=1000, n_features=4,
#                            n_informative=2, n_redundant=0,
#                            random_state=0, shuffle=False)
clf = RandomForestClassifier(n_estimators=100, max_depth=12,
                             random_state=0)
clf.fit(x_train, y_train)  
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=12, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)
print(clf.feature_importances_)